### Question 1

Interpretability is important in machine learning because it helps us understand how a model makes its decisions. This is especially important in sensitive areas like healthcare, finance, or hiring, where decisions can have a big impact on people's lives. Being able to explain a model's predictions builds trust, makes it easier to catch and fix biases or errors, and helps ensure the model is being used fairly and responsibly. It also supports legal and ethical standards, giving users a way to understand or even challenge decisions made by the model. Without interpretability, we risk relying on “black box” models that make decisions we can't fully understand or justify.

### Question 2

In [ ]:
!pip install autofeat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd
from autofeat import FeatureSelector

# load data
X, y = load_diabetes(return_X_y=True, as_frame=True)

# initialize FeatureSelector
fs = FeatureSelector(verbose=1)
X_selected = fs.fit_transform(X, y)

# number of discarded features
num_original_features = X.shape[1]
num_selected_features = X_selected.shape[1]
num_discarded = num_original_features - num_selected_features

print(f"Original features: {num_original_features}")
print(f"Selected features: {num_selected_features}")
print(f"Discarded features: {num_discarded}")

[featsel] Scaling data...done.
Original features: 10
Selected features: 6
Discarded features: 4


/usr/local/lib/python3.11/dist-packages/autofeat/featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


### Question 3

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# use selected features
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# fit a regression model
model = LinearRegression()
model.fit(X_train, y_train)

# evaluate
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))

print(f"R2 score (train): {r2_train:.4f}")
print(f"R2 score (test): {r2_test:.4f}")

R2 score (train): 0.5213
R2 score (test): 0.4632


### Question 4

In [ ]:
from autofeat import AutoFeatRegressor

# initialize and fit AutoFeatRegressor with 3 feature engineering steps
afr = AutoFeatRegressor(verbose=1, feateng_steps=3)
X_train_feat = afr.fit_transform(X_train, y_train)
X_test_feat = afr.transform(X_test)

model.fit(X_train_feat, y_train)

[featsel] Scaling data...done.


/usr/local/lib/python3.11/dist-packages/autofeat/featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LinearRegression()

In [ ]:
# calculate r^2 scores
r2_train_feat = model.score(X_train_feat, y_train)
r2_test_feat = model.score(X_test_feat, y_test)

print(f"R² score on training set (with feature engineering): {r2_train_feat:.4f}")
print(f"R² score on test set (with feature engineering): {r2_test_feat:.4f}")

# show 5 new features generated by AutoFeat
new_features = afr.new_feat_cols_[:5]
print("Five new features generated:", new_features)

R² score on training set (with feature engineering): 0.5746
R² score on test set (with feature engineering): 0.5072
Five new features generated: ['Abs(s5)/s5', 'exp(s3)*Abs(sex)', 'exp(bmi)*exp(bp)', 'exp(bmi)*exp(s5)', 's3/Abs(bmi)']


The AutoFeatRegressor output shows a clear improvement in both training and test $R^2$ scores, rising to 0.5746 and 0.5072 respectively, compared to the baseline model. While this suggests that the engineered features captured more complex patterns, the gap between training and test performance indicates a possible risk of overfitting. The newly created features, such as exp(bmi)*exp(s5) and s3/Abs(bmi), reflect non-linear relationships that may boost model accuracy but should be used with caution to ensure generalizability.